# Developing AI Systems with the OpenAI API - Part 1

## Structuring End-to-End Applications

Learn how to seamlessly incorporate the OpenAI API into more complex systems, mastering the art of manipulating the response format and adeptly handling errors. This chapter provides essential knowledge for those looking to build components of AI systems that are not just interactive but also robust and efficient.

In [ ]:
import os

# Set your OpenAI API key
openai_api_key = os.environ['OPENAI_API_KEY']

### Formatting model response as JSON
As a librarian cataloging new books, you aim to leverage the OpenAI API to automate the creation of a JSON file from text notes you received from a colleague. Your task is to extract relevant information such as book titles and authors and to do this, you use the OpenAI API to convert the text notes, that include book titles and authors, into structured JSON files.

In this and all the following exercises, the openai library has already been loaded. Entering your own API key is not necessary to create requests and complete the exercises in this course; however, you may do so if you prefer.

In [ ]:
from openai import OpenAI

# Create the OpenAI client
client = OpenAI(api_key=openai_api_key)

# Create the request
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", 
         "content": "I have these notes with book titles and authors: New releases this week! The Beholders by Hester Musson, The Mystery Guest by Nita Prose. Please organize the titles and authors in a json file."}
    ],
    # Specify the response format
    response_format={"type": "json_object"}
)

# Print the response
print(response.choices[0].message.content)


{
  "new_releases": [
    {
      "title": "The Beholders",
      "author": "Hester Musson"
    },
    {
      "title": "The Mystery Guest",
      "author": "Nita Prose"
    }
  ]
}


### Handling exceptions
You are working at a logistics company on developing an application that uses the OpenAI API to check the shipping address of your top three customers. The application will be used internally and you want to make sure that other teams are presented with an easy to read message in case of error.

To address this requirement, you decide to print a custom message in case the users fail to provide a valid key for authentication, and use a try and except block to handle that.

The message variable has already been imported.

In [9]:
from openai import AuthenticationError, RateLimitError


client = OpenAI(api_key='abcd')

# Use the try statement
try:
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "user", 
             "content": "I have these notes with book titles and authors: New releases this week! The Beholders by Hester Musson, The Mystery Guest by Nita Prose. Please organize the titles and authors in a json file."}
        ],
        # Specify the response format
        response_format={"type": "json_object"}
    )

    print(response.choices[0].message.content)
# Use the except statement
except AuthenticationError as e:
    print("Please double check your authentication key and try again, the one provided is not valid.")
except RateLimitError as e:
    print(f"OpenAI API request exceeded rate limit: {e}")
except Exception as e:
    print(f"Unable to generate a response. Exception: {e}")

Please double check your authentication key and try again, the one provided is not valid.


### Avoiding rate limits with retry
You've created a function to run Chat Completions with a custom message but have noticed it sometimes fails due to rate limits. You decide to use the @retry decorator from the tenacity library to avoid errors when possible.

In [12]:
# Import the tenacity library
from tenacity import retry, wait_random_exponential, stop_after_attempt

client = OpenAI(api_key=openai_api_key)

# Add the appropriate parameters to the decorator
@retry(wait=wait_random_exponential(min=5, max=40), stop=stop_after_attempt(4))
def get_response(messages, model="gpt-4o-mini"):
    response = client.chat.completions.create(
        model=model,
        messages=messages
    )
    return response.choices[0].message.content

print(get_response([{"role": "user", "content": "List ten holiday destinations."}], 
                   "gpt-4o-mini"))

Sure! Here are ten popular holiday destinations:

1. Bali, Indonesia
2. Paris, France
3. Tokyo, Japan
4. Santorini, Greece
5. New York City, USA
6. Rome, Italy
7. Sydney, Australia
8. Marrakech, Morocco
9. Cancun, Mexico
10. Cape Town, South Africa

These destinations offer a mix of culture, adventure, relaxation, and stunning scenery.


### Batching messages
You are developing a fitness application to track running and cycling training, but find out that all your customers' distances have been measured in kilometers, and you'd like to have them also converted to miles.

You decide to use the OpenAI API to send requests for each measurement, but want to avoid using a for loop that would send too many requests. You decide to send the requests in batches, specifying a system message that asks to convert each of the measurements from kilometers to miles and present the results in a table containing both the original and converted measurements.

The measurements list (containing a list of floats) and the get_response() function have already been imported.

In [ ]:
measurements = [5.2, 6.3, 3.7]

messages = []
# Provide a system message and user messages to send the batch
messages.append({"role": "system", 
                 "content": "Convert each of the measurements from kilometers to miles and present the results in a table containing both the original and converted measurements."})
# Append measurements to the message
[messages.append({"role": "user", "content": str(i)}) for i in measurements]

response = get_response(messages)
print(response)

To convert kilometers to miles, you can use the conversion factor where 1 kilometer is approximately equal to 0.621371 miles. Here are the converted measurements presented in a table:

| Kilometers | Miles    |
|------------|----------|
| 5.2        | 3.228    |
| 6.3        | 3.913    |
| 3.7        | 2.298    |

### Conversion calculations:
- 5.2 km * 0.621371 = 3.228 miles
- 6.3 km * 0.621371 = 3.913 miles
- 3.7 km * 0.621371 = 2.298 miles


### Setting token limits
An e-commerce platform just hired you to improve the performance of their customer service bot built using the OpenAI API. You've decided to start by ensuring that the input messages do not cause any rate limit issue by setting a limit of 100 tokens, and test it with a sample input message.

The tiktoken library has been preloaded.

In [14]:
import tiktoken

input_message = {"role": "user", 
                 "content": "I'd like to buy a shirt and a jacket. Can you suggest two color pairings for these items?"}

# Use tiktoken to create the encoding for your model
encoding = tiktoken.encoding_for_model("gpt-4o-mini")
# Check for the number of tokens
num_tokens = len(encoding.encode(input_message['content']))

# Run the chat completions function and print the response
if num_tokens <= 100:
    response = client.chat.completions.create(model="gpt-4o-mini", messages=[input_message])
    print(response.choices[0].message.content)
else:
    print("Message exceeds token limit")

Sure! Here are two color pairings for a shirt and a jacket:

1. **Navy Blue Jacket with a White Shirt**: This classic combination is versatile and perfect for both casual and smart-casual occasions. You can add jeans or chinos to complete the look.

2. **Olive Green Jacket with a Light Gray Shirt**: This pairing offers a modern and stylish vibe. The earthy tone of the olive green jacket complements the neutral light gray shirt, creating a balanced and fresh outfit.

Feel free to mix and match based on your personal style!
